In [1]:
import os
import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
data_dir = "/root/20242R0136COSE47402/FinalProject/data/test"
class_candidate = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
text_inputs = []
for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):
        folder = folder.replace('_', ' ')
        text_inputs.append(f"a photo of {folder}")

In [14]:
image_paths = []
image_labels = []

for class_name in class_candidate:
    class_folder = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_folder):
        if img_name[0] == '.':
            continue
        img_path = os.path.join(class_folder, img_name)
        image_paths.append(img_path)
        class_name = class_name.replace('_', ' ')
        image_labels.append(f"a photo of {class_name}")
        

print(f"test dataset size : {len(image_paths)}")

test dataset size : 25250


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [9]:
ans = 0
model.eval()

for idx, img_path in enumerate(image_paths):
    image = Image.open(img_path).convert('RGB')
    inputs = processor(text=text_inputs, images=image, return_tensors='pt', padding=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    if text_inputs[torch.argmax(probs).item()] == image_labels[idx]:
        ans += 1
    if idx % 100 == 0:
        print(f"# {idx} finished.")

precision = ans / len(image_paths)
precision


UnidentifiedImageError: cannot identify image file '/root/20242R0136COSE47402/FinalProject/data/test/foie_gras/._1335868.jpg'

In [ ]:
model.eval()

outputs = model(**inputs)

outputs.keys()

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [ ]:
logits_per_image = outputs.logits_per_image
logits_per_image.shape
probs = logits_per_image.softmax(dim=1)

import torch
candidates[torch.argmax(outputs.logits_per_text.softmax(dim=0))]

'a zebra is looking.'